In [8]:
from tensorflow.keras.layers import Activation , concatenate , Dense
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_addons as tfa

In [9]:
input_shape = (160, 120, 3)
learning_rate = 0.001
weight_decay = 0.0001
batch_size = 256
num_epochs = 50
image_size = 256  # We'll resize input images to this size
patch_size = 32  # Size of the patches to be extract from the input images
num_patches = (image_size // patch_size) ** 2
projection_dim = 64
num_heads = 4
transformer_units = [
    projection_dim * 2,
    projection_dim,
]  # Size of the transformer layers
transformer_layers = 2
mlp_head_units = [2048, 1024]  # Size of the dense layers of the final classifier

In [10]:
data_augmentation = keras.Sequential(
    [
        layers.Normalization(),
        layers.Resizing(image_size, image_size),
        
    ],
    name="data_augmentation",
)


In [11]:
def mlp(x, hidden_units, dropout_rate):
    for units in hidden_units:
        x = layers.Dense(units, activation=tf.nn.gelu)(x)
        x = layers.Dropout(dropout_rate)(x)
    return x


In [12]:
class Patches(layers.Layer):
    def __init__(self, patch_size):
        super().__init__()
        self.patch_size = patch_size

    def call(self, images):
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(
            images=images,
            sizes=[1, self.patch_size, self.patch_size, 1],
            strides=[1, self.patch_size, self.patch_size, 1],
            rates=[1, 1, 1, 1],
            padding="VALID",
        )
        patch_dims = patches.shape[-1]
        patches = tf.reshape(patches, [batch_size, -1, patch_dims])
        return patches

In [13]:
class PatchEncoder(layers.Layer):
    def __init__(self, num_patches, projection_dim):
        super().__init__()
        self.num_patches = num_patches
        self.projection = layers.Dense(units=projection_dim) #Linear Projection Of Patch-Tockens
        self.position_embedding = layers.Embedding(
            input_dim=num_patches, output_dim=projection_dim
        )

    def call(self, patch):
        positions = tf.range(start=0, limit=self.num_patches, delta=1)
        encoded = self.projection(patch) + self.position_embedding(positions)
        return encoded

In [14]:
def create_vit_classifier():
    image_inputs = layers.Input(shape=input_shape) #shape=(None, 32, 32, 3)
    goal_inputs = layers.Input(shape=(2,))
    # Augment data.
    augmented = data_augmentation(image_inputs) #shape=(None, 72, 72, 3)
    # Create patches.
    patches = Patches(patch_size)(augmented) #shape=(None, None, 108)
    
    #Add Goal to Patches
    Goal = mlp(goal_inputs,[32,512,3072],dropout_rate=0.05) #shape=(1, 192)
    Goal = tf.reshape(Goal,[1,1,3072])

    tf.keras.layers.Concatenate(axis=1)([Goal,patches])    
    # Encode patches.
    encoded_patches = PatchEncoder(num_patches, projection_dim)(patches) #shape=(None, 144, 64)
    
    
    # Create multiple layers of the Transformer block.
    for _ in range(transformer_layers):
        # Layer normalization 1.
        x1 = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
        # Create a multi-head attention layer.
        attention_output = layers.MultiHeadAttention( #Self Attention mechanism
            num_heads=num_heads, key_dim=projection_dim, dropout=0.1
        )(x1, x1)
        # Skip connection 1.
        x2 = layers.Add()([attention_output, encoded_patches])
        # Layer normalization 2.
        x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
        # MLP.
        x3 = mlp(x3, hidden_units=transformer_units, dropout_rate=0.1)
        # Skip connection 2.
        encoded_patches = layers.Add()([x3, x2])

        
        
        
        # Create a [batch_size, projection_dim] tensor.
        representation = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
        representation = layers.Flatten()(representation)
        representation = layers.Dropout(0.3)(representation)
        #so far we created the image representation
    
        # Add MLP.
        features = mlp(representation, hidden_units=mlp_head_units, dropout_rate=0.5)
        print(features.shape)
    
    # Classify outputs.##############################################################################################
    
    linear_vel = layers.Dense(512, activation="relu" , name = 'first_lin')(features)
    linear_vel = layers.Dense(128, activation="relu")(linear_vel)
    linear_vel = layers.Dense(32, activation="relu")(linear_vel)
    linear_vel = layers.Dense(1,)(linear_vel)                        
    linear_net =Activation("linear", name="linear_output")(linear_vel)                  
     
     
                                                                                         
    angular_vel = layers.Dense(512, activation="relu",name = 'first_ang')(features)
    angular_vel = layers.Dense(128, activation="relu")(angular_vel)
    angular_vel = layers.Dense(32, activation="relu")(angular_vel)
    angular_vel = layers.Dense(1,)(linear_vel)                        
    angular_net =Activation("linear", name="angular_output")(angular_vel)
                                                                                         
    
    
    # Create the Keras model.##############################################################################
    model = keras.Model(inputs=[image_inputs,goal_inputs], outputs=[linear_net,angular_net])             
    return model

In [17]:
def Target():
    image_inputs = layers.Input(shape=input_shape) #shape=(None, 32, 32, 3)
    goal_inputs = layers.Input(shape=(2,))
    # Augment data.
    augmented = data_augmentation(image_inputs) #shape=(None, 72, 72, 3)
    # Create patches.
    patches = Patches(patch_size)(augmented) #shape=(None, None, 108)
    
    #Add Goal to Patches
    Goal = mlp(goal_inputs,[32,512,3072],dropout_rate=0.05) #shape=(1, 192)
    Goal = tf.reshape(Goal,[1,1,3072])

    tf.keras.layers.Concatenate(axis=1)([Goal,patches])    
    # Encode patches.
    encoded_patches = PatchEncoder(num_patches, projection_dim)(patches) #shape=(None, 144, 64)
    
    
    # Create multiple layers of the Transformer block.
    for _ in range(transformer_layers):
        # Layer normalization 1.
        x1 = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
        # Create a multi-head attention layer.
        attention_output = layers.MultiHeadAttention( #Self Attention mechanism
            num_heads=num_heads, key_dim=projection_dim, dropout=0.1
        )(x1, x1)
        # Skip connection 1.
        x2 = layers.Add()([attention_output, encoded_patches])
        # Layer normalization 2.
        x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
        # MLP.
        x3 = mlp(x3, hidden_units=transformer_units, dropout_rate=0.1)
        # Skip connection 2.
        encoded_patches = layers.Add()([x3, x2])

        
        
        
        # Create a [batch_size, projection_dim] tensor.
        representation = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
        representation = layers.Flatten()(representation)
        representation = layers.Dropout(0.3)(representation)
        #so far we created the image representation
    
        # Add MLP.
        features = mlp(representation, hidden_units=mlp_head_units, dropout_rate=0.5)
        print(features.shape)
    
    # Classify outputs.##############################################################################################
    """
    linear_vel = layers.Dense(512, activation="relu" , name = 'first_lin')(features)
    linear_vel = layers.Dense(128, activation="relu")(linear_vel)
    linear_vel = layers.Dense(32, activation="relu")(linear_vel)
    linear_vel = layers.Dense(1,)(linear_vel)                        
    linear_net =Activation("linear", name="linear_output")(linear_vel)                  
     
     
                                                                                         
    angular_vel = layers.Dense(512, activation="relu",name = 'first_ang')(features)
    angular_vel = layers.Dense(128, activation="relu")(angular_vel)
    angular_vel = layers.Dense(32, activation="relu")(angular_vel)
    angular_vel = layers.Dense(1,)(linear_vel)                        
    angular_net =Activation("linear", name="angular_output")(angular_vel)
                                                                                         
    """
    
    # Create the Keras model.##############################################################################
    model = keras.Model(inputs=[image_inputs,goal_inputs], outputs=[features])             
    return model

In [18]:
def updateTargetModel_actor(model=None,target=None):
        weights= model.get_weights()
        target_net = target.get_weights()
        
        for i in range(44,53):

            new_weight = weights[i]
            target_net[i-44] = new_weight
 
        target.set_weights(target_net)
 
        return target

In [23]:
def updateTargetModel_back(model=None,target=None):
        weights= model.get_weights()
        target_net = target.get_weights()
        
        for i in range(len(target_net)):

            new_weight = weights[i]
            target_net[i] = new_weight
 
        target.set_weights(target_net)
 
        return target

In [20]:
def updateTargetModel_reward(model=None,target=None):
        weights= model.get_weights()
        target_net = target.get_weights()
        
        for i in range(len(weights)):

            new_weight = weights[i]
            target_net[i] = new_weight
 
        target.set_weights(target_net)
 
        return target

In [21]:
def actor():
    features = layers.Input(shape=(1024,))
    
    linear_vel = layers.Dense(512, activation="relu" , name = 'first_lin')(features)
    linear_vel = layers.Dense(128, activation="relu")(linear_vel)
    linear_vel = layers.Dense(32, activation="relu")(linear_vel)
    linear_vel = layers.Dense(1,)(linear_vel)                        
    linear_net =Activation("linear", name="linear_output")(linear_vel)                  
     
     
                                                                                         
    angular_vel = layers.Dense(512, activation="relu",name = 'first_ang')(features)
    angular_vel = layers.Dense(128, activation="relu")(angular_vel)
    angular_vel = layers.Dense(32, activation="relu")(angular_vel)
    angular_vel = layers.Dense(1,)(linear_vel)                        
    angular_net =Activation("linear", name="angular_output")(angular_vel)
    
    model = keras.Model(inputs = features , outputs = [linear_vel,angular_vel])
    
    return model

In [29]:
def trainable_reward_func(in_Dims=1024):
    features= keras.Input(shape=(in_Dims,))
    
    linear_vel = layers.Dense(512, activation="relu" , name = 'first_lin')(features)
    linear_vel = layers.Dense(128, activation="relu")(linear_vel)
    linear_vel = layers.Dense(32, activation="relu")(linear_vel)
    linear_vel = layers.Dense(1,)(linear_vel)                        
    linear_net =Activation("linear", name="linear_output")(linear_vel)                  
     
     
                                                                                         
    angular_vel = layers.Dense(512, activation="relu",name = 'first_ang')(features)
    angular_vel = layers.Dense(128, activation="relu")(angular_vel)
    angular_vel = layers.Dense(32, activation="relu")(angular_vel)
    angular_vel = layers.Dense(1,)(linear_vel)                        
    angular_net =Activation("linear", name="angular_output")(angular_vel)
    
    ConCat = concatenate([linear_net , angular_net] , axis = -1)
    
    x=Dense(32,activation="relu")(ConCat)
    x=Dense(16,activation="relu")(x)
    x=Dense(1)(x)
    
    out=Activation("linear",name="rew_out")(x)
    
    
    reward_net = keras.Model(inputs=features,outputs=out,name="TPr_net")
    
    return reward_net

In [ ]:
log_dir = 'D:\\Ali_Reza\\kir to hossein\\model\\'
checkpoint_filepath = log_dir + 'ep120-loss0.195-val_loss0.314.h5'
model = create_vit_classifier()
model.load_weights(checkpoint_filepath)
print('[info] .... weight loaded')

In [25]:
back_bone = Target()
backBone = updateTargetModel_back(model = model , target = back_bone)
path_1 = log_dir + 'Back_bone.h5'
backBone.save_weights(path_1)

(None, 1024)
(None, 1024)


In [26]:
path_2 = log_dir + 'actor.h5'
actor = actor()
Actor = updateTargetModel_actor(model=model,target=actor)
Actor.save_weights(path_2)

In [30]:
path_3 = log_dir + 'reward.h5'
reward=trainable_reward_func()
reward_net = updateTargetModel_reward(model=Actor,target=reward)
reward_net.save_weights(path_3)   

In [40]:
reward_net = updateTargetModel_2(tau=1,model=acc,target=reward)

In [41]:
checkpoint_filepath_3 = log_dir + 'part3_new.h5'
reward_net.save_weights(checkpoint_filepath_3)